In [1]:
import os

import numpy as np
import pandas as pd
import networkx as nx

import arrow
from tqdm import tqdm

from dotenv import load_dotenv
from coinbase.wallet.client import Client

load_dotenv('.env')
client = Client(os.environ['COINBASE_KEY'], os.environ['COINBASE_SECRET'])

### Change here to select project output

In [2]:
# projects = ['bayc', 'coolcats', 'cryptoadz', 'cyberkongz', 'hashmasks', 'mayc', 'meebits', 'mekaverse', 'svs']
projects = ['cryptoadz', 'cyberkongz']

### Store base data as a dataframe

In [3]:
def create_base_data(project):
    PATH_TO_DATA = './data/collated/' + project + '.csv'  # Change if needed
    column_names = ["row", "tx_hash", "token_address", "from_address", "to_address", "token_id", "blk_number", "blk_timestamp", "eth_value"]
    
    df = pd.read_csv(PATH_TO_DATA, delimiter=',', skiprows=1, names=column_names)
    
    df["from_address"] = df.from_address.apply(lambda x: x.strip())
    df["to_address"] = df.to_address.apply(lambda x: x.strip())
    
    return df

### Transaction data

In [4]:
def get_transaction_data(project):
    PATH_TO_DATA = f"./data/balances/{project}.csv"
    return pd.read_csv(PATH_TO_DATA)

errors = []

def lookup_transaction_value(df, block, account):
    value = 0
    
    if account == '0x0000000000000000000000000000000000000000':
        return value
    
    try:
        b = df[(df['block'] == block) & (df['address'] == account)]
        value = b['eth_value'].head(1).iat[0]
    except Exception as e:
        errors.append((block, account))
    return value

# Convert ETH value to USD at specified date
prices = {}
def get_usd_value(date, eth_value):
    if eth_value == 0:
        return eth_value

    if date in prices:
        return float(prices[date]) * eth_value
    else:
        prices[date] = client.get_spot_price(currency_pair='ETH-USD', date=date)['amount']
        return float(prices[date]) * eth_value

### Build time-based dataframes

In [5]:
def create_timed_data(df, df_transactions):
    ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
    column_names = [
        "date", 
        "days_since_mint", 
        "from_address", 
        "to_address", 
        "token_id", 
        "blk_number", 
        "eth_value",
        #"usd_value",
        "from_value", 
        "to_value",
    ]
    
    df_time = pd.DataFrame(columns=column_names)
    df_total = df.shape[0]
    
    for index, row in tqdm(df.iterrows(), total=df_total):
        blk_timestamp = row['blk_timestamp']
        date = arrow.get(blk_timestamp).datetime

        from_address = row['from_address']
        to_address = row['to_address']
        token_id = row['token_id']
        blk_number = row['blk_number']
        eth_value = row['eth_value']
        
        if from_address == ZERO_ADDRESS:
            days_since_mint = 0
            mint_date = date
        else:
            days_since_mint = date - mint_date
            
        #usd_value = get_usd_value(date, eth_value)
            
        from_value = lookup_transaction_value(df_transactions, blk_number, from_address)
        to_value = lookup_transaction_value(df_transactions, blk_number, to_address)
            
        df_time = df_time.append({
            'date': date,
            'days_since_mint': days_since_mint,
            'from_address': from_address,
            'to_address': to_address,
            'token_id': token_id, 
            'blk_number': blk_number,
            'eth_value': eth_value,
            #'usd_value': usd_value,
            'from_value': from_value,
            'to_value': to_value,
        }, ignore_index=True)

    return df_time

### Build graph objects from time base dataframes

In [6]:
def build_graph_from_timed(df_time):    
    # Building a network per block
    # we will use a weighted and directed graph.
    graph = nx.MultiDiGraph()

    # loop over the pandas dataframe.
    for index, row in tqdm(df_time.iterrows(), total=df_time.shape[0]):
        # read the values from the dataframe.
        # token_id  blk_timestamp eth_value 
        date = row['date']
        from_address = row['from_address']
        to_address = row['to_address']
        token_id = row['token_id']
        blk_number = row['blk_number']
        eth_value = row['eth_value']
        #usd_value = row['usd_value']
        from_value = row['from_value']
        to_value = row['to_value']
        
        # make sure both addresses are in the graph.
        if from_address not in graph:
            graph.add_node(from_address)
        if to_address not in graph:
            graph.add_node(to_address)

        # set the attributes on this node.
        nx.set_node_attributes(graph, {from_address: from_value, to_address: to_value}, 'eth_value')

        # keep track of how many trades a wallet has done.
        trades = nx.get_node_attributes(graph, "trades")
        if from_address in trades:
            nx.set_node_attributes(graph, {from_address:trades[from_address] + 1}, 'trades')
        else:
            nx.set_node_attributes(graph, {from_address:1}, 'trades')
        if to_address in trades:
            nx.set_node_attributes(graph, {to_address:trades[to_address] + 1}, 'trades')
        else:
            nx.set_node_attributes(graph, {to_address:1}, 'trades')

        # check if this NFT has already been sold and if yes, remove the old sale.
        # this might be a candidate for memoization - c.b.
        remove_edges = []
        for (u,v,d) in graph.edges.data():
            if d['token_id'] == token_id:
                remove_edges.append((u,v))
        # we need to remove them in a seperate step, since otherwise we change the datastructure that we are iterating over.
        for (u,v) in remove_edges:
            graph.remove_edge(u,v)

        # add an edge for the transaction.
        graph.add_edge(from_address, to_address, weight=eth_value, token_id=token_id) # keep track of token id by adding it to the edge.
        
    return graph

### Build time-based snapshots

In [7]:
def build_graph_snapshots(df_time):
    """Simpler version of below function which just returns the graph objects"""
    res = {}
    
    df_time['date_quantile'], bins = pd.qcut(df_time['date'], 10, labels=False, retbins=True)
    time_buckets = np.unique(df_time["date_quantile"].to_numpy())
    
    for time_bucket, label in zip(time_buckets, bins):
        selection = df_time[(df_time['date_quantile'] == time_bucket)]
        graph_snapshot = build_graph_from_timed(selection)
        res[label] = graph_snapshot
        
    return res

In [8]:
def build_df_snapshots(df_time):
    column_names = [
        "time_bucket", 
        "time_bucket_label",
        "number_of_nodes", 
        "avg_clustering", 
        "reciprocity", 
        "assortativity", 
        "assortativity_base", 
        "assortativity_out_out", 
        "assortativity_in_in", 
        "assortativity_in_out",
        "centrality_degree",
        "centrality_closeness", 
        "centrality_betweenness",
        "centrality_eigenvector",
        "avg_clustering_random",
        "assortativity_random"
    ]
    
    df_snapshots = pd.DataFrame(columns=column_names)
    
    df_time['date_quantile'], bins = pd.qcut(df_time['date'], 10, labels=False, retbins=True)
    time_buckets = np.unique(df_time["date_quantile"].to_numpy())
    
    for time_bucket, label in zip(time_buckets, bins):
        selection = df_time[(df_time['date_quantile'] == time_bucket)]
        graph_snapshot = build_graph_from_timed(selection)
        
        df_snapshots = df_snapshots.append({
            "time_bucket": time_bucket,
            "time_bucket_label": label,
            "number_of_nodes": graph_snapshot.number_of_nodes(),
            "reciprocity": nx.reciprocity(graph_snapshot),
            "assortativity": nx.degree_assortativity_coefficient(graph_snapshot),
            "assortativity_base": nx.degree_pearson_correlation_coefficient(graph_snapshot.to_undirected(), weight='weight'),
            "assortativity_out_out": nx.degree_pearson_correlation_coefficient(graph_snapshot, x='out', y='out', weight='weight'),
            "assortativity_in_in": nx.degree_pearson_correlation_coefficient(graph_snapshot, x='in', y='in', weight='weight'),
            "assortativity_in_out": nx.degree_pearson_correlation_coefficient(graph_snapshot, x='in', y='out', weight='weight'),
            "centrality_degree": nx.degree_centrality(graph_snapshot),
            "centrality_closeness": nx.closeness_centrality(graph_snapshot),
        }, ignore_index=True)
        
    return df_snapshots.sort_values(by=['time_bucket'])

In [9]:
for project in projects:
    df_transactions = get_transaction_data(project)
    df_time = create_timed_data(create_base_data(project), df_transactions)
    print(len(errors))
    
    g_time = build_graph_from_timed(df_time)
    
    np.save(f"./memory/{project}/full.npy", df_time)
    nx.write_gml(g_time, f"./memory/{project}/full.gml")

100%|█████████████████████████████████████| 29120/29120 [05:09<00:00, 94.23it/s]


51215


100%|████████████████████████████████████| 15189/15189 [01:39<00:00, 153.04it/s]


76917


100%|████████████████████████████████████| 15189/15189 [00:23<00:00, 653.64it/s]


In [11]:
for project in projects:
    column_names = [
        "date", 
        "days_since_mint", 
        "from_address", 
        "to_address", 
        "token_id", 
        "blk_number", 
        "eth_value",
        "from_value", 
        "to_value",
    ]
    
    np_data = np.load(f"./memory/{project}/full.npy", allow_pickle=True)
    df_time = pd.DataFrame(data=np_data, columns=column_names)
    
    g_snapshots = build_graph_snapshots(df_time)
    df_snapshot_summary = build_df_snapshots(df_time)
    
    for i, snapshot in enumerate(g_snapshots.keys()):
        nx.write_gml(g_time, f"./memory/{project}/snapshots/{i}.gml")
    
    np.save(f"./memory/{project}/snapshots/summary.npy", df_time)

100%|█████████████████████████████████████| 3114/3114 [00:01<00:00, 2174.75it/s]
/Users/corey/uzh/HS21/network/venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
100%|█████████████████████████████████████| 2779/2779 [00:01<00:00, 2473.45it/s]
/Users/corey/uzh/HS21/network/venv/lib/python3.8/site-packages/networkx/algorithms/assortativity/correlation.py:282: RuntimeWarning: invalid value encountered in double_scalars
  return (xy * (M - ab)).sum() / np.sqrt(vara * varb)
/Users/corey/uzh/HS21/network/venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
100%|█████████████████████████████████████| 1519/1519 [00:00<00:00, 2362.82it/s]


In [12]:
print(get_transaction_data('cyberkongz'))

         row     block        date  \
0          0  12253142  2021-04-16   
1          1     block        date   
2          2  12253527  2021-04-16   
3          3  12253570  2021-04-16   
4          4  12253684  2021-04-16   
...      ...       ...         ...   
21230  21230  13711959  2021-11-30   
21231  21231  13711968  2021-11-30   
21232  21232  13711968  2021-11-30   
21233  21233  13711969  2021-11-30   
21234  21234  13711969  2021-11-30   

                                          address             eth_value  \
0      0x000000048797808be86aa9786c1f402671192d6b             1.3410686   
1                                         address             eth_value   
2      0x721931508df2764fd4f70c53da646cb8aed16ace  1.231442803923159847   
3      0x721931508df2764fd4f70c53da646cb8aed16ace  1.193960158923159847   
4      0xfa6c54de608c9a0a2c2a3220bb7e42b95d1b910b  0.473936397986028232   
...                                           ...                   ...   
21230  0xa145d8108